In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

cookies = {
   #Aquí irían tus cookies
}

headers = {
    #'User-Agent': 'Aquí iría tu User-Agent, he borrado el mío ',
    'Accept': '*/*',
    'Accept-Language': 'es-CL,es;q=0.8,en-US;q=0.5,en;q=0.3',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'Origin': 'https://cead.spd.gov.cl',
    'Connection': 'keep-alive',
    'Referer': 'https://cead.spd.gov.cl/estadisticas-delictuales/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
}

# Datos fijos que se mantendrán para todas las iteraciones
base_data = {
    'medida': '1',
    'tipoVal': '1,2',
    'anio[]': [
        '2023',
        '2022',
        '2021',
        '2020',
        '2019',
        '2018',
        '2017',
        '2016',
        '2015',
        '2014',
        '2013',
        '2012',
        '2011',
        '2010',
        '2009',
        '2008',
        '2007',
        '2006',
        '2005',
    ],
    'trimestre[]': [
        '1',
        '2',
        '3',
        '4',
    ],
    'mes[]': [
        '1',
        '2',
        '3',
        '4',
        '5',
        '6',
        '7',
        '8',
        '9',
        '10',
        '11',
        '12',
    ],
    'mes_nombres[]': [
        'Enero',
        'Febrero',
        'Marzo',
        'Abril',
        'Mayo',
        'Junio',
        'Julio',
        'Agosto',
        'Septiembre',
        'Octubre',
        'Noviembre',
        'Diciembre',
    ],
    'seleccion': '2',
    'descarga': 'false',
}

# Lista de regiones con sus códigos
regiones = {
    'Tarapacá': '1',
    'Antofagasta': '2',
    'Atacama': '3',
    'Coquimbo': '4',
    'Valparaíso': '5',
    'Lib. Bernardo O\'Higgins': '6',
    'Maule': '7',
    'Biobío': '8',
    'Araucanía': '9',
    'Los Lagos': '10',
    'Aysén': '11',
    'Magallanes': '12',
    'Metropolitana': '13',
    'Los Ríos': '14',
    'Arica y Parinacota': '15',
    'Ñúble': '16',
}

# Lista de delitos y grupos
delitos = [
    {'nombre': 'Abandono de armas', 'codigo': '24', 'grupo': '5'},
    {'nombre': 'Abigeato', 'codigo': '1', 'grupo': '1'},
    {'nombre': 'Abusos sexuales y otros delitos sexuales', 'codigo': '2', 'grupo': '2'},
    {'nombre': 'Amenazas', 'codigo': '15', 'grupo': '4'},
    {'nombre': 'Comercio ambulante o clandestino', 'codigo': '16', 'grupo': '4'},
    {'nombre': 'Consumo alcohol vía pública', 'codigo': '17', 'grupo': '4'},
    {'nombre': 'Daños', 'codigo': '18', 'grupo': '4'},
    {'nombre': 'Desórdenes', 'codigo': '19', 'grupo': '4'},
    {'nombre': 'Ebriedad', 'codigo': '20', 'grupo': '4'},
    {'nombre': 'Hallazgo de armas o explosivos', 'codigo': '25', 'grupo': '5'},
    {'nombre': 'Homicidios', 'codigo': '3', 'grupo': '3'},
    {'nombre': 'Hurtos', 'codigo': '4', 'grupo': '3'},
    {'nombre': 'Lesiones leves', 'codigo': '5', 'grupo': '3'},
    {'nombre': 'Lesiones menos graves, graves o gravísimas', 'codigo': '6', 'grupo': '3'},
    {'nombre': 'Otras incivilidades', 'codigo': '21', 'grupo': '4'},
    {'nombre': 'Otros ley de armas', 'codigo': '26', 'grupo': '5'},
    {'nombre': 'Otros robos con fuerza', 'codigo': '7', 'grupo': '3'},
    {'nombre': 'Porte de armas', 'codigo': '27', 'grupo': '5'},
    {'nombre': 'Receptación', 'codigo': '29', 'grupo': '6'},
    {'nombre': 'Riña pública', 'codigo': '22', 'grupo': '4'},
    {'nombre': 'Robo con violencia o intimidación', 'codigo': '8', 'grupo': '3'},
    {'nombre': 'Robo de objetos de o desde vehículo', 'codigo': '9', 'grupo': '3'},
    {'nombre': 'Robo de vehículo motorizado', 'codigo': '10', 'grupo': '3'},
    {'nombre': 'Robo en lugar habitado', 'codigo': '11', 'grupo': '3'},
    {'nombre': 'Robo en lugar no habitado', 'codigo': '12', 'grupo': '3'},
    {'nombre': 'Robo frustrado', 'codigo': '30', 'grupo': '7'},
    {'nombre': 'Robo por sorpresa', 'codigo': '13', 'grupo': '3'},
    {'nombre': 'Ruidos molestos', 'codigo': '23', 'grupo': '4'},
    {'nombre': 'Tenencia ilegal de armas o explosivos', 'codigo': '28', 'grupo': '5'},
    {'nombre': 'Violaciones', 'codigo': '14', 'grupo': '3'},
    {'nombre': 'Violencia intrafamiliar a adulto mayor', 'codigo': '31', 'grupo': '8'},
    {'nombre': 'Violencia intrafamiliar a hombre', 'codigo': '32', 'grupo': '8'},
    {'nombre': 'Violencia intrafamiliar a mujer', 'codigo': '33', 'grupo': '8'},
    {'nombre': 'Violencia intrafamiliar a niño', 'codigo': '34', 'grupo': '8'},
    {'nombre': 'Violencia intrafamiliar no clasificado', 'codigo': '35', 'grupo': '8'},
]

# Lista que contendrá todos los DataFrames creados para cada combinación de delito y grupo
dfs = []

# Iterar sobre cada región
for region, codigo_region in regiones.items():
    # Iterar sobre cada delito y grupo
    for delito in delitos:
        data = base_data.copy()
        data['delitos[]'] = delito['codigo']
        data['delitos_nombres[]'] = delito['nombre']
        data['delitos_agrupados[]'] = delito['grupo']
        data['region[]'] = codigo_region
        data['region_nombres[]'] = region

        # Realizar la solicitud HTTP
        response = requests.post(
            'https://cead.spd.gov.cl/wp-content/themes/gobcl-wp-master/data/get_estadisticas_delictuales.php',
            cookies=cookies,
            headers=headers,
            data=data,
        )

        # Parsear la respuesta HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontrar la tabla en el HTML
        table = soup.find('table')

        # Convertir la tabla en un DataFrame con la opción thousands
        df_list = pd.read_html(str(table), thousands='.', converters={'REGION': str})

        # Agregar el nombre de la región como una columna en cada DataFrame
        for df in df_list:
            df['Region'] = region

        # Agregar el DataFrame a la lista
        dfs.extend(df_list)

# Concatenar todos los DataFrames en uno solo
result_df = pd.concat(dfs, ignore_index=True)

display(result_df)


Unnamed: 0_level_0   2005                        \
                            Unnamed: 0_level_1 TRIM 1           TRIM 2        
                      GRUPO DELICTUAL / DELITO      1    2    3      4    5   
0                    Infracción a ley de armas     10    7   12     11   21   
1                            Abandono de armas      0    0    0      0    0   
2                                     Abigeato      1    0    0      0    0   
3     Abusos sexuales y otros delitos sexuales     16    9    7      9   11   
4                                Incivilidades    511  439  503    476  496   
...                                        ...    ...  ...  ...    ...  ...   
1051           Violencia intrafamiliar a mujer    233  194  188    194  203   
1052                   Violencia intrafamiliar    254  221  221    215  236   
1053            Violencia intrafamiliar a niño      5    4    6      4    5   
1054                   Violencia intrafamiliar    254  221  221    215  236   
1055    Violencia intrafamiliar no clasificado      0    0    0      0    0   

                            ...   2023                                         \
          TRIM 3            ... TRIM 2           TRIM 3           TRIM 4        
        6      7    8    9  ...      4    5    6      7    8    9     10   11   
0      15     19   12   22  ...     91   89   71     73   69   77     54   45   
1       0      0    0    0  ...      0    0    0      0    0    0      0    0   
2       0      0    1    0  ...      0    0    0      0    1    0      0    0   
3      10      8   18    9  ...     30   39   28     24   42   35     24   44   
4     477    629  560  640  ...    673  675  681    762  718  729    741  698   
...   ...    ...  ...  ...  ...    ...  ...  ...    ...  ...  ...    ...  ...   
1051  188    164  160  176  ...    208  224  197    228  222  208    209  197   
1052  217    191  186  205  ...    279  301  283    288  297  272    293  280   
1053    6      6    4    5  ...      4    4    6      9    6    7      8    8   
1054  217    191  186  205  ...    279  301  283    288  297  272    293  280   
1055    0      0    0    0  ...     13   14   11     13   13    8     18   14   

             Region  
                     
       12            
0      52  Tarapacá  
1       0  Tarapacá  
2       0  Tarapacá  
3      33  Tarapacá  
4     662  Tarapacá  
...   ...       ...  
1051  216     Ñúble  
1052  299     Ñúble  
1053    9     Ñúble  
1054  299     Ñúble  
1055   13     Ñúble  

[1056 rows x 230 columns]

In [2]:
result_df

Unnamed: 0_level_0   2005                        \
                            Unnamed: 0_level_1 TRIM 1           TRIM 2        
                      GRUPO DELICTUAL / DELITO      1    2    3      4    5   
0                    Infracción a ley de armas     10    7   12     11   21   
1                            Abandono de armas      0    0    0      0    0   
2                                     Abigeato      1    0    0      0    0   
3     Abusos sexuales y otros delitos sexuales     16    9    7      9   11   
4                                Incivilidades    511  439  503    476  496   
...                                        ...    ...  ...  ...    ...  ...   
1051           Violencia intrafamiliar a mujer    233  194  188    194  203   
1052                   Violencia intrafamiliar    254  221  221    215  236   
1053            Violencia intrafamiliar a niño      5    4    6      4    5   
1054                   Violencia intrafamiliar    254  221  221    215  236   
1055    Violencia intrafamiliar no clasificado      0    0    0      0    0   

                            ...   2023                                         \
          TRIM 3            ... TRIM 2           TRIM 3           TRIM 4        
        6      7    8    9  ...      4    5    6      7    8    9     10   11   
0      15     19   12   22  ...     91   89   71     73   69   77     54   45   
1       0      0    0    0  ...      0    0    0      0    0    0      0    0   
2       0      0    1    0  ...      0    0    0      0    1    0      0    0   
3      10      8   18    9  ...     30   39   28     24   42   35     24   44   
4     477    629  560  640  ...    673  675  681    762  718  729    741  698   
...   ...    ...  ...  ...  ...    ...  ...  ...    ...  ...  ...    ...  ...   
1051  188    164  160  176  ...    208  224  197    228  222  208    209  197   
1052  217    191  186  205  ...    279  301  283    288  297  272    293  280   
1053    6      6    4    5  ...      4    4    6      9    6    7      8    8   
1054  217    191  186  205  ...    279  301  283    288  297  272    293  280   
1055    0      0    0    0  ...     13   14   11     13   13    8     18   14   

             Region  
                     
       12            
0      52  Tarapacá  
1       0  Tarapacá  
2       0  Tarapacá  
3      33  Tarapacá  
4     662  Tarapacá  
...   ...       ...  
1051  216     Ñúble  
1052  299     Ñúble  
1053    9     Ñúble  
1054  299     Ñúble  
1055   13     Ñúble  

[1056 rows x 230 columns]

In [3]:
result_df.to_excel('dbdelitosfinal.xlsx')

print("Archivo Excel generado correctamente.")

Archivo Excel generado correctamente.


In [4]:
# Guardar el DataFrame resultante en un archivo CSV
result_df.to_csv('dbdelitosfinal.csv')

print("Archivo CSV generado correctamente.")


Archivo CSV generado correctamente.
